## **Notebook #9**
## Encoder-Decoder LSTM for Natural Language Processing.
**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Arthur Felipe dos Santos Fernandes


In [3]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [4]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for _ in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [27]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [6]:
from math import ceil
from math import log10

In [7]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [28]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [29]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [10]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [11]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [30]:
X,y = integer_encode(X,y,alphabet)

In [31]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [14]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
        
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [32]:
X,y = one_hot_encode(X,y,len(alphabet))

In [34]:
print(X)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [17]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [18]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [19]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [20]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [21]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [22]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 75)            0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            612       
 ibuted)                                                         
                                                                 
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 [==============================] - 30s 4ms/step - loss: 0.4068 - accuracy: 0.8763


In [24]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.027486, Accuracy: 100.000000


In [25]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+7+5 = 15 (expect 15)
  2+10+6 = 18 (expect 18)
   4+5+9 = 18 (expect 18)
   2+2+4 =  8 (expect  8)
 10+7+10 = 27 (expect 27)
  10+2+3 = 15 (expect 15)
   8+4+1 = 13 (expect 13)
   8+1+3 = 12 (expect 12)
   5+9+6 = 20 (expect 20)
   1+5+8 = 14 (expect 14)


###  Explique como são gerados os dados de treinamento
Os dados são gerados a partir da função generate_data, que engloba um conjunto de tarefas distribuídos em funções menores, a começar com random_sum_pairs que gera aleatoriamente as operações de soma e seus respectivos resultados; em seguida a função to_string converte essas operações em strings e adiciona identação; a função integer_encode converte cada caractere da string em um inteiro único enquanto que a função one_hot_encode converte esses valores em um vetor one hot ( 1 para o elemento apresentado e 0 para todos os demais)

### Explique como uma calculadora simples pode operar baseada no conceito de tradução de frase
A ideia consiste no fato de que cada elemento de que uma operação aciona uma trajetória no espaço semântico do bloco LSTM, portanto, o valor correspondente da operação é equivalente ao ponto final atingido pela trajetória no espaço semântico.